In [1]:
import requests
import re
import pandas as pd
import xml.etree.ElementTree as ET
import json
import os
import sys
import matplotlib.pyplot as plt
from collections import Counter
import xml.etree.ElementTree as ET
from typing import List, Dict, Any, Optional
import duckdb
import unicodedata
import importlib

from pathlib import Path
import re
import pandas as pd

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [2]:
import grela_helpers

In [4]:
conn = grela_helpers.make_connection()

In [5]:
grela_helpers.init_grela_streams(conn)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
#grela_extraction_helpers.conn.close()
# importlib.reload(grela_extraction_helpers)

In [6]:
# establish connection with Google sheets
file_data = json.load(open(os.path.expanduser("../../../ServiceAccountsKey.json")))
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(
    ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

ruland_gs = gc.open_by_url(
    "https://docs.google.com/spreadsheets/d/1od248fJpNbwMO8IVZQVO3xnzrmmTe68yS9XWajyuk5o/edit?usp=sharing")

In [7]:
lexeme_df = pd.read_parquet("../data/ruland-dictionaries.parquet")

In [8]:
lexeme_df["target_len"] = lexeme_df["target_canonical"].str.split().str.len()

In [9]:
lexeme_df = lexeme_df[lexeme_df["target_relemmatized"].str.split().str.len() < 4]

In [10]:
MAX_N = 3
lexeme_df = lexeme_df[lexeme_df["target_len"] <= MAX_N].copy()
len(lexeme_df)

2846

In [11]:
# look at a few multiword entries
lexeme_df[lexeme_df["target_len"] > 1].head(5)

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized
23,Lapis maior,[Lapis maior],lapis maior,"[Ruland1612-Lapis-maior, Ruland1612-Lapis-maio...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Lapis-m...",3,"[finde nach etlicher Meynung, die calcinirten ...","[None, None, id est, der Geist außgezogen auß ...","[finde nach etlicher Meynung, die calcinirten ...","[{'entry_id': 'Ruland1612-Lapis-maior', 'quote...",2,lapis magnus
24,Leo viridis,[Leo viridis],leo uiridis,"[Ruland1612-Leo-viridis, Ruland1612-Leo-viridi...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Leo-vir...",3,"[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[None, quorundam opinione aurum, vitriolum]","[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[{'entry_id': 'Ruland1612-Leo-viridis', 'quote...",2,leo uiridis
25,Machina tractoria,[Machina tractoria],machina tractoria,"[Ruland1612-Machina-tractoria, Ruland1612-Mach...",3,[M],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Machina...",3,"[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[None, None, None]","[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[{'entry_id': 'Ruland1612-Machina-tractoria', ...",2,machina tractoria
30,Panum aereorum species,[Panum aereorum species],panum aereorum species,"[Ruland1612-Panum-aereorum-species-,-Vorpleits...",3,[P],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Panum-a...",3,"[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...","[None, None, None]","[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...",[{'entry_id': 'Ruland1612-Panum-aereorum-speci...,3,panis aereus species
32,Pyritae aurei coloris,[Pyritae aurei coloris],pyritae aurei coloris,[Ruland1612-Pyritae-aurei-coloris-.-1.-Pyrites...,3,[P],[],[],"[{'def': '1', 'entry_id': 'Ruland1612-Pyritae-...",3,"[1, è quibus ignis elicitur, è quibus ignis no...","[1, è quibus ignis elicitur, è quibus ignis no...",[1. Pyrites qui est solidus & colorem auri tot...,[{'entry_id': 'Ruland1612-Pyritae-aurei-colori...,3,pyritus aureus color


In [12]:
# look at entries starting with mercurius
lexeme_df[lexeme_df["target_canonical"].str.startswith("mercurius")]

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized
27,Mercurius,[Mercurius],mercurius,"[Ruland1612-Mercurius, Ruland1612-Mercurius, R...",3,[M],[],[],"[{'def': 'id est, sulphur.', 'entry_id': 'Rula...",3,"[id est, sulphur., est principium materiale, v...","[id est, sulphur., est principium materiale, v...","[id est, sulphur. Mercurius. Mercurius ist in ...","[{'entry_id': 'Ruland1612-Mercurius', 'quote':...",1,mercurius
96,Mercurius metallorum,[Mercurius metallorum],mercurius metallorum,"[Ruland1612-Mercurius-metallorum, Ruland1612-M...",2,[M],[],[],[{'def': 'Ist darauß die Natur der Coͤrper gez...,2,[Ist darauß die Natur der Coͤrper gezogen wird...,[Ist darauß die Natur der Coͤrper gezogen wird...,[Ist darauß die Natur der Coͤrper gezogen wird...,[{'entry_id': 'Ruland1612-Mercurius-metallorum...,2,mercurius metallum
1880,Mercurius argentipigmentum,[Mercurius argentipigmentum],mercurius argentipigmentum,[Ruland1612-Mercurius-argentipigmentum],1,[M],[],[],"[{'def': 'ist Schwefel victriol, Alaun Saltz-d...",1,"[ist Schwefel victriol, Alaun Saltz-dieweils d...","[ist Schwefel victriol, Alaun Saltz-dieweils d...","[ist Schwefel victriol, Alaun Saltz-dieweils d...",[{'entry_id': 'Ruland1612-Mercurius-argentipig...,2,mercurius argentipigmentum
1881,Mercurius chambar,[Mercurius chambar],mercurius chambar,[Ruland1612-Mercurius-chambar],1,[M],[],[],"[{'def': 'ist magnesia, fuessend Coͤrper und W...",1,"[ist magnesia, fuessend Coͤrper und Wasser.]","[ist magnesia, fuessend Coͤrper und Wasser.]","[ist magnesia, fuessend Coͤrper und Wasser.]","[{'entry_id': 'Ruland1612-Mercurius-chambar', ...",2,mercurius chaambar
1882,Mercurius corallinus,[Mercurius corallinus],mercurius corallinus,[Ruland1612-Mercurius-corallinus],1,[M],[],[],[{'def': 'qui per oleum ouorum & aquas alias i...,1,[qui per oleum ouorum & aquas alias in rubedin...,[qui per oleum ouorum & aquas alias in rubedin...,[qui per oleum ouorum & aquas alias in rubedin...,[],2,mercurius corallinus
1883,Mercurius crudus,[Mercurius crudus],mercurius crudus,[Ruland1612-Mercurius-crudus],1,[M],[],[],"[{'def': 'est is, qui nondum separatus est a s...",1,"[est is, qui nondum separatus est a sua matric...","[est is, qui nondum separatus est a sua matric...","[est is, qui nondum separatus est a sua matric...","[{'entry_id': 'Ruland1612-Mercurius-crudus', '...",2,mercurius crudus
1884,Mercurius crystallinus,[Mercurius crystallinus],mercurius crystallinus,"[Ruland1612-Mercurius-crystallinus,]",1,[M],[],[],[{'def': 'qui saepe sublimatus est in formam c...,1,[qui saepe sublimatus est in formam crystalli ...,[qui saepe sublimatus est in formam crystalli ...,[qui saepe sublimatus est in formam crystalli ...,[],2,mercurius crystallinus
1885,Mercurius laxus,[Mercurius laxus],mercurius laxus,[Ruland1612-Mercurius-laxus],1,[M],[],[],"[{'def': 'est turbith minerale.', 'entry_id': ...",1,[est turbith minerale.],[est turbith minerale.],[est turbith minerale.],[],2,mercurius laxus
1886,Mercurius mineralium,[Mercurius mineralium],mercurius mineralium,[Ruland1612-Mercurius-mineralium],1,[M],[],[],"[{'def': 'est oleitas, vel vnctuositas de mine...",1,"[est oleitas, vel vnctuositas de mineris auri ...","[est oleitas, vel vnctuositas de mineris auri ...","[est oleitas, vel vnctuositas de mineris auri ...",[{'entry_id': 'Ruland1612-Mercurius-mineralium...,2,mercurius mineralis
1887,Mercurius regeneratus,[Mercurius regeneratus],mercurius regeneratus,[Ruland1612-Mercurius-regeneratus],1,[M],[],[],"[{'def': 'est primum ens Mercurii.', 'entry_id...",1,[est primum ens Mercurii.],[est primum ens Mercurii.],[est primum ens Mercurii.],[],2,mercurius regenero


In [14]:
# test with one specific multiword phrase
target_canonical     =  "lapis philosophorum" # "lapis philosophorum"
target_relemmatized  =  "lapis philosophus" # "lapis philosophus"

df = grela_helpers.concordance_for_target_across_sentences(
    conn,
    target_canonical=target_canonical,
    target_relemmatized=target_relemmatized,
    window=10,
    emlap_only=True,
    heavy_normalization=False,
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [15]:
df

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens
0,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352412024, 352412025]","[10, 11]","[17, 18]",emlap_100002,emlap_100002_162,emlap_100002_162,emlap_100002_162,", aquarum tam uegetabilium quam mineralium ad ...","[{'token_id': 352412014, 'token_text': ',', 'l...","Et cum hoc sufficit tibi modus generalis , aqu...","[{'token_id': 352412007, 'token_text': 'Et', '..."
1,"[lapidis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352425603, 352425604]","[10, 11]","[3, 4]",emlap_100002,emlap_100002_750,emlap_100002_749,emlap_100002_750,quia ibi large continetur hec doctrina . ¶item...,"[{'token_id': 352425593, 'token_text': 'quia',...",¶item in compositione lapidis philosophorum ru...,"[{'token_id': 352425600, 'token_text': '¶item'..."
2,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352426859, 352426860]","[10, 11]","[3, 4]",emlap_100002,emlap_100002_800,emlap_100002_798,emlap_100002_801,"& uirtutem sue nature . ¶questio . Pater , si ...","[{'token_id': 352426849, 'token_text': '&', 'l...","Pater , si lapis philosophorum cadit in doctri...","[{'token_id': 352426856, 'token_text': 'Pater'..."
3,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352426881, 352426882]","[10, 11]","[14, 15]",emlap_100002,emlap_100002_801,emlap_100002_801,emlap_100002_801,"non solum ad preciosos lapides , sed et iam ad...","[{'token_id': 352426871, 'token_text': 'non', ...","Fili , hec regula non solum ad preciosos lapid...","[{'token_id': 352426867, 'token_text': 'Fili',..."
4,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[353269572, 353269573]","[10, 11]","[3, 4]",emlap_100004,emlap_100004_10754,emlap_100004_10752,emlap_100004_10756,ordinem . Incipiunt capitula primi libri . Quo...,"[{'token_id': 353269562, 'token_text': 'ordine...",Quod unus est lapis philosophorum Cap .,"[{'token_id': 353269569, 'token_text': 'Quod',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,"[Lapidem, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[353749437, 353749438]","[10, 11]","[0, 1]",emlap_100099,emlap_100099_217,emlap_100099_216,emlap_100099_217,optimis medicamentis ad de pellendos morbos id...,"[{'token_id': 353749427, 'token_text': 'optimi...",Lapidem Philosophorum seu tincturam & Medicina...,"[{'token_id': 353749437, 'token_text': 'Lapide..."
463,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[353749577, 353749578]","[10, 11]","[3, 4]",emlap_100099,emlap_100099_227,emlap_100099_226,emlap_100099_227,"Moyses praeparauerit , pauci Chemici intelligu...","[{'token_id': 353749567, 'token_text': 'Moyses...","Ueterum scriptis de Lapide Philosophorum , eiu...","[{'token_id': 353749574, 'token_text': 'Ueteru..."
464,"[lapidem, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[353746822, 353746823]","[10, 11]","[4, 5]",emlap_100099,emlap_100099_67,emlap_100099_66,emlap_100099_67,si singulari De Ibeneficio adiuti . Posse aute...,"[{'token_id': 353746812, 'token_text': 'si', '...",Posse autem Chemicos talem lapidem Philosophor...,"[{'token_id': 353746818, 'token_text': 'Posse'..."
465,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[354310339, 354310340]","[10, 11]","[8, 9]",emlap_100100,emlap_100100_33988,emlap_100100_33987,emlap_100100_33988,uixerit . Imo neque unquam facile inducar ut c...,"[{'token_id': 354310329, 'token_text': 'uixeri...","Imo neque unquam facile inducar ut credam , la...","[{'token_id': 354310331, 'token_text': 'Imo', ..."


In [17]:
target_canonical     =  "mercurius metallorum" # "lapis philosophorum"
target_relemmatized  =  "mercurius metallum" # "lapis philosophus"

df = grela_helpers.concordance_for_target_across_sentences(
    conn,
    target_canonical=target_canonical,
    target_relemmatized=target_relemmatized,
    window=10,
    emlap_only=True,
    heavy_normalization=False,
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [18]:
df

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens
0,"[Mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[353223376, 353223377]","[10, 11]","[53, 54]",emlap_100004,emlap_100004_8365,emlap_100004_8365,emlap_100004_8366,"omnibus modis praedominatur ignis , & illud di...","[{'token_id': 353223366, 'token_text': 'omnibu...","Uiuum causat metalla , quanuis unum differt ab...","[{'token_id': 353223323, 'token_text': 'Uiuum'..."
1,"[mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[354467395, 354467396]","[10, 11]","[53, 54]",emlap_100007,emlap_100007_1504,emlap_100007_1504,emlap_100007_1506,"omnibus moribus predominatur ignis , & illud d...","[{'token_id': 354467385, 'token_text': 'omnibu...",Uiuum causat metalla quamuis adhuc bene differ...,"[{'token_id': 354467342, 'token_text': 'Uiuum'..."
2,"[mercurius, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[350152161, 350152162]","[10, 11]","[28, 29]",emlap_100010,emlap_100010_470,emlap_100010_470,emlap_100010_470,"similitudine uerae calcis communis , quia arge...","[{'token_id': 350152151, 'token_text': 'simili...",Compraehendis igitur ex supradictis rationibus...,"[{'token_id': 350152133, 'token_text': 'Compra..."
3,"[mercurium, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[350154009, 350154010]","[10, 11]","[6, 7]",emlap_100010,emlap_100010_569,emlap_100010_568,emlap_100010_569,"sulphure uerba faciemus . Prius diximus , per ...","[{'token_id': 350153999, 'token_text': 'sulphu...","Prius diximus , per calcinationem , mercurium ...","[{'token_id': 350154003, 'token_text': 'Prius'..."
4,"[Mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356302332, 356302333]","[10, 11]","[54, 55]",emlap_100011,emlap_100011_3877,emlap_100011_3877,emlap_100011_3878,"omnibus modis praedominatur ignis , & illud di...","[{'token_id': 356302322, 'token_text': 'omnibu...","Uiuum causat metalla , quamuis unum differt ab...","[{'token_id': 356302278, 'token_text': 'Uiuum'..."
5,"[Mercurii, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356161082, 356161083]","[10, 11]","[1, 2]",emlap_100016,emlap_100016_6506,emlap_100016_6504,emlap_100016_6508,Quomodo argentum uiuum in sulphur conuertatur ...,"[{'token_id': 356161072, 'token_text': 'Quomod...",137 Mercurii metallorum quomodo differant .,"[{'token_id': 356161081, 'token_text': '137', ..."
6,"[Mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356165596, 356165597]","[10, 11]","[57, 58]",emlap_100022,emlap_100022_141,emlap_100022_141,emlap_100022_142,"omnibus moribus praedominatur ignis , & illud ...","[{'token_id': 356165586, 'token_text': 'omnibu...","Uiuum causat metalla , quamuis adhuc bene diff...","[{'token_id': 356165539, 'token_text': 'Uiuum'..."
7,"[Mercurium, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[356170017, 356170018]","[10, 11]","[13, 14]",emlap_100022,emlap_100022_326,emlap_100022_326,emlap_100022_326,", tunc sequeretur , quod de nouo extra primam ...","[{'token_id': 356170007, 'token_text': ',', 'l...","Si esset possibile , tunc sequeretur , quod de...","[{'token_id': 356170004, 'token_text': 'Si', '..."
8,"[mercurium, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[354376074, 354376075]","[10, 11]","[16, 17]",emlap_100029,emlap_100029_529,emlap_100029_529,emlap_100029_530,"eorum , quae isti minoralia uocant , quintam e...","[{'token_id': 354376064, 'token_text': 'eorum'...",Obseruandum igitur in Elixiris insequentibus s...,"[{'token_id': 354376058, 'token_text': 'Obseru..."
9,"[mercurius, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[354328567, 354328568]","[10, 11]","[2, 3]",emlap_100033,emlap_100033_624,emlap_100033_623,emlap_100033_624,mercurii praeparatio hec omnia in sese contine...,"[{'token_id': 354328557, 'token_text': 'me

In [19]:
df["kwic_tokens"].tolist()[0]

array([{'token_id': 353223366, 'token_text': 'omnibus', 'lemma': 'omnis', 'pos': 'DET', 'ref': '{"page": [233], "textblock": [9], "tag": "", "blocktype": "text"}', 'sentence_id': 'emlap_100004_8365', 'char_start': 252, 'char_end': 259},
       {'token_id': 353223367, 'token_text': 'modis', 'lemma': 'modus', 'pos': 'NOUN', 'ref': '{"page": [233], "textblock": [9], "tag": "", "blocktype": "text"}', 'sentence_id': 'emlap_100004_8365', 'char_start': 260, 'char_end': 265},
       {'token_id': 353223368, 'token_text': 'praedominatur', 'lemma': 'praedominor', 'pos': 'VERB', 'ref': '{"page": [233], "textblock": [9], "tag": "", "blocktype": "text"}', 'sentence_id': 'emlap_100004_8365', 'char_start': 266, 'char_end': 279},
       {'token_id': 353223369, 'token_text': 'ignis', 'lemma': 'ignis', 'pos': 'NOUN', 'ref': '{"page": [233], "textblock": [9], "tag": "", "blocktype": "text"}', 'sentence_id': 'emlap_100004_8365', 'char_start': 280, 'char_end': 285},
       {'token_id': 353223370, 'token_tex

In [20]:
lexeme_df_sample_emlap_instances = lexeme_df.sample(10, random_state=1).apply(lambda row: grela_helpers.concordance_for_target_across_sentences(
    conn,
    row["target_canonical"],
    row["target_relemmatized"],
    window=10,
    emlap_only=True,
    heavy_normalization=False,
).to_dict("records"), axis=1)
lexeme_df_sample_emlap_instances

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

1868                                                   []
1134    [{'target_phrase': ['descensum'], 'target_from...
1626                                                   []
1690    [{'target_phrase': ['lapidibus', 'calcem'], 't...
2037    [{'target_phrase': ['Organopeotica'], 'target_...
1030    [{'target_phrase': ['contractionem'], 'target_...
635                                                    []
1826                                                   []
1631                                                   []
2381    [{'target_phrase': ['Scoria'], 'target_from': ...
dtype: object

In [28]:
outdir = Path("../data/large_files/emlap_ruland_instances/")
outdir.mkdir(exist_ok=True)

In [29]:
def safe_name(s: str | None) -> str:
    """Make a safe short filename component from a target phrase."""
    if not s or not isinstance(s, str):
        return "unknown"
    s = s.strip().lower().replace(" ", "_")
    s = re.sub(r"[^a-z0-9_]+", "", s)   # keep only safe chars
    return s or "unnamed"

In [30]:
lexeme_df["instance_fname"] = lexeme_df["target_canonical"].apply(lambda x: safe_name(x) + ".parquet")

In [31]:
lexeme_df[lexeme_df.duplicated(subset="instance_fname", keep=False)]

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized,instance_fname


In [32]:
done = []

In [26]:
tc = "mercurius metallorum"
tr = "mercurius metallum"
name = safe_name(tc)
fname = f"{name}.parquet"
fpath = str(outdir / fname)
grela_helpers.concordance_for_target_across_sentences(
    conn,
    tc,
    tr,
    window=10,
    out_path=fpath,
    emlap_only=True,
    heavy_normalization=False,
)

In [ ]:
for i, row in lexeme_df.iterrows():
    tc = row.get("target_canonical")
    tr = row.get("target_relemmatized")
    target = tc or tr
    name = safe_name(tc)
    fname = f"{name}.parquet"
    if fname not in done:
        fpath = str(outdir / fname)
            # streams directly to Parquet (no df = ..., no df.to_parquet)
        grela_helpers.concordance_for_target_across_sentences(
            conn,
            tc,
            tr,
            window=10,
            out_path=fpath,
            emlap_only=True,
            heavy_normalization=False,
        )
        print(f"[{i}] saved → {fpath}")
        done.append(fname)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[0] saved → ../data/large_files/emlap_ruland_instances/naphtha.parquet
[1] saved → ../data/large_files/emlap_ruland_instances/baurac.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[2] saved → ../data/large_files/emlap_ruland_instances/nitrum.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

[130] saved → ../data/large_files/emlap_ruland_instances/uena.parquet
[131] saved → ../data/large_files/emlap_ruland_instances/uena_cumulata.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[132] saved → ../data/large_files/emlap_ruland_instances/uenus.parquet
[133] saved → ../data/large_files/emlap_ruland_instances/albertus.parquet
[134] saved → ../data/large_files/emlap_ruland_instances/cinnabaris.parquet
[137] saved → ../data/large_files/emlap_ruland_instances/abacus.parquet
[138] saved → ../data/large_files/emlap_ruland_instances/abacus_maior.parquet
[139] saved → ../data/large_files/emlap_ruland_instances/abartamen.parquet
[140] saved → ../data/large_files/emlap_ruland_instances/abesamum.parquet
[141] saved → ../data/large_files/emlap_ruland_instances/abessi.parquet
[142] saved → ../data/large_files/emlap_ruland_instances/abesum.parquet
[143] saved → ../data/large_files/emlap_ruland_instances/abicum.parquet
[144] saved → ../data/large_files/emlap_ruland_instances/ablatio.parquet
[145] saved → ../data/large_files/emlap_ruland_instances/abluiren.parquet
[146] saved → ../data/large_files/emlap_ruland_instances/ablutio.parquet
[147] saved → ../data/large_files/emlap_rula

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[159] saved → ../data/large_files/emlap_ruland_instances/acetum.parquet
[160] saved → ../data/large_files/emlap_ruland_instances/acetum_amineum.parquet
[161] saved → ../data/large_files/emlap_ruland_instances/acetum_philosophorum.parquet
[162] saved → ../data/large_files/emlap_ruland_instances/acetum_radicale.parquet
[163] saved → ../data/large_files/emlap_ruland_instances/acetum_radicatum_nonnullis.parquet
[164] saved → ../data/large_files/emlap_ruland_instances/acies.parquet
[165] saved → ../data/large_files/emlap_ruland_instances/acordina.parquet
[166] saved → ../data/large_files/emlap_ruland_instances/acortinus.parquet
[167] saved → ../data/large_files/emlap_ruland_instances/acsuo.parquet
[168] saved → ../data/large_files/emlap_ruland_instances/acureb.parquet
[169] saved → ../data/large_files/emlap_ruland_instances/acus.parquet
[170] saved → ../data/large_files/emlap_ruland_instances/acusto.parquet
[171] saved → ../data/large_files/emlap_ruland_instances/adamas_acuminatus.parquet
[

In [ ]:
len(os.listdir(outdir))

In [54]:
#lexeme_df["emlap_instances"] = lexeme_df.apply(lambda row: concordance_for_target_across_sentences(conn, row["target_canonical"], row["target_relemmatized"], window=10).to_dict("records"), axis=1)

In [ ]:
def read_hits(fname):
    try:
        path = os.path.join(outdir, fname)
        instances = pd.read_parquet(path).to_dict("records")
    except:
        instances = []
    return instances
lexeme_df["emlap_instances"] = lexeme_df["instance_fname"].apply(read_hits)

In [ ]:
lexeme_df["emlap_instances_N"] = lexeme_df["emlap_instances"].apply(len)

In [ ]:
lexeme_df.sort_values("emlap_instances_N", ascending=False)[:20]

In [ ]:
lexeme_df["instances_ids"] = lexeme_df["emlap_instances"].apply(lambda x: [ins["grela_id"][6:] for ins in x])

In [ ]:
emlap_metadata = pd.read_csv(
        "https://raw.githubusercontent.com/CCS-ZCU/EMLAP_ETL/refs/heads/master/data/emlap_metadata.csv",
        sep=";",
    )
emlap_metadata.head(5)

In [ ]:
emlap_id_title_dict = dict(zip(emlap_metadata['no.'].astype(str), emlap_metadata['working_title']))
emlap_id_date_dict = dict(zip(emlap_metadata['no.'].astype(str), emlap_metadata['date_publication']))

In [ ]:
def add_emlap_metadata(instances_dict_list):
    instances_dict_list_new = []
    for instance_dict in instances_dict_list:
        id = instance_dict["grela_id"][6:]
        instance_dict["title"] = emlap_id_title_dict[id]
        instance_dict["year"] = emlap_id_date_dict[id]
        instances_dict_list_new.append(instance_dict)
    return instances_dict_list_new

In [ ]:
# add metadata to instances column
lexeme_df["emlap_instances"] = lexeme_df["emlap_instances"].apply(add_emlap_metadata)

In [ ]:
# export instances also into CSV...
csv_path = "../data/emlap_ruland_instances_csv/"
os.makedirs(csv_path, exist_ok=True)

def create_csvs(fname):
        path = os.path.join(outdir, fname)
        instances_df = pd.read_parquet(path)
        print(len(instances_df))
        #instances_dict_list = instances_df.to_dict("records")
        #instances_dict_list_new = add_emlap_metadata(instances_dict_list)
        #instances_df_new = pd.DataFrame(instances_dict_li# st_new)
        fpath = os.path.join(csv_path, fname.replace(".parquet", ".csv"))
        #instances_df_new.to_csv(fpath)
        instances_df.to_csv(fpath)

In [ ]:
for fname in lexeme_df["instance_fname"]:
    create_csvs(fname)

In [32]:
lexeme_df.head(5)

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized,instance_fname,emlap_instances,emlap_instances_N,instances_ids
0,Naphtha,[Naphtha],naphtha,[Ruland1612-Naphtha],1,[N],[],[],"[{'def': 'id est, pir, ignis.; id est, flatus ...",15,"[id est, pir, ignis.; id est, flatus minerae, ...","[id est, pir, ignis.; id est, flatus minerae, ...",[Iudaicum bitumen diximus esse speciem Naphtha...,"[{'entry_id': 'Ruland1612-Naphtha', 'quote': '...",1,naphtha,naphtha.parquet,"[{'target_phrase': ['naphtha'], 'target_from':...",6,"[100012, 100012, 100012, 100038, 100063, 100070]"
1,Baurac,"[Baurac, Bäurac]",baurac,"[Ruland1612-Baurac, Ruland1612-Baurac, Ruland1...",10,[B],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Baurac'...",10,"[wird vom Schaum deß Glases oder Nitere., id e...","[None, id est, sal gemmae., i. e. id est sapph...","[wird vom Schaum deß Glases oder Nitere., id e...","[{'entry_id': 'Ruland1612-Baurac', 'quote': 'w...",1,baurac,baurac.parquet,[],0,[]
2,Nitrum,[Nitrum],nitrum,"[None, Ruland1612-Nitrum, Ruland1612-Nitrum, R...",9,[N],[],[],"[{'def': 'Baurach, sal petrosum, nitrum à Germ...",9,"[Baurach, sal petrosum, nitrum à German, Fossi...","[Baurach, sal petrosum, nitrum à German, Fossi...","[Baurach, sal petrosum, nitrum à German. Salpe...","[{'entry_id': None, 'quote': 'SalpeterBergsalt...",1,nitrum,nitrum.parquet,"[{'target_phrase': ['Nitra'], 'target_from': '...",488,"[100001, 100001, 100002, 100002, 100005, 10000..."
3,Elixir,[Elixir],elixir,"[Ruland1612-Elixir,-Elei,, Ruland1612-Elixir, ...",6,[E],"[Elei, Xir]",[],"[{'def': 'id est, Medicina.', 'entry_id': 'Rul...",5,"[id est, Medicina., est fermentum, cuius minim...","[id est, Medicina., est fermentum, cuius minim...","[id est, Medicina., est fermentum, cuius minim...","[{'entry_id': 'Ruland1612-Elixir', 'quote': 'i...",1,elixir,elixir.parquet,"[{'target_phrase': ['Elixir'], 'target_from': ...",1064,"[100001, 100007, 100007, 100007, 100007, 10000..."
4,Magnesia,[Magnesia],magnesia,"[Ruland1612-Magnesia, Ruland1612-Magnesia, Rul...",5,[M],[],[],"[{'def': 'id est, testudo vel sulphul.', 'entr...",5,"[id est, testudo vel sulphul., Ist das gem sch...","[id est, testudo vel sulphul., Ist das gem sch...","[id est, testudo vel sulphul., Ist das gem sch...","[{'entry_id': 'Ruland1612-Magnesia', 'quote': ...",1,magnesia,magnesia.parquet,"[{'target_phrase': ['Magnesia'], 'target_from'...",222,"[100003, 100003, 100003, 100003, 100007, 10000..."


In [ ]:
# conservative under Google’s 50k hard limit
MAX_CELL = 48000

def _clip(s: str, n: int = MAX_CELL) -> str:
    return s if len(s) <= n else (s[:n-1] + "…")

def _stringify(v) -> str:
    """Controlled stringification so lists/dicts don't explode."""
    if v is None or (isinstance(v, float) and np.isnan(v)):
        return ""
    if isinstance(v, (list, dict)):
        # compact JSON, unicode-friendly
        return json.dumps(v, ensure_ascii=False, separators=(',', ':'))
    return str(v)

def prepare_for_sheets(df: pd.DataFrame, *, drop_heavy: bool = False) -> pd.DataFrame:
    """
    Make a sheets-safe copy:
      - optional: drop very heavy/nested cols
      - stringify lists/dicts in a compact way
      - clip any cell > MAX_CELL
    """
    if drop_heavy:
        df = df.drop(columns=[
            # keep this list lean; tweak as you like
            "senses_flat", "Senses", "Translations", "EntryLevelTranslations"
        ], errors="ignore").copy()
    else:
        df = df.copy()

    # stringify then clip per-cell
    df = df.applymap(_stringify)
    df = df.applymap(lambda s: _clip(s, MAX_CELL))
    return df

# --- example usage ---
# keep everything but serialize/clip:
df_for_sheet = prepare_for_sheets(lexeme_df.drop(["emlap_instances"], axis=1), drop_heavy=False)

In [ ]:
set_with_dataframe(ruland_gs.add_worksheet("ruland_emlap_instances_2025-12-12", 1,1), df_for_sheet.sort_values("emlap_instances_N", ascending=False))

In [ ]:
lexeme_df.to_json("../data/large_files/ruland-emlap-grela.json")

In [ ]:
lexeme_df.to_parquet("../data/large_files/ruland-emlap-grela.parquet")

In [ ]:
outdir_fullgrela = ("../data/large_files/grela_ruland_instances/")

In [ ]:
for i, row in lexeme_df.iterrows():
    tc = row.get("target_canonical")
    tr = row.get("target_relemmatized")
    target = tc or tr
    name = safe_name(tc)
    fname = f"{name}.parquet"
    if fname not in done:
        fpath = str(outdir_fullgrela / fname)
            # streams directly to Parquet (no df = ..., no df.to_parquet)
        grela_helpers.concordance_for_target_across_sentences(
            conn,
            tc,
            tr,
            window=10,
            out_path=fpath,
            emlap_only=False,
            heavy_normalization=False,
        )
        print(f"[{i}] saved → {fpath}")
        done.append(fname)